In [1]:
import pandas as pd
import math

In [2]:
#Read two files and merge
df1 = pd.read_csv('../data/ipo_list', sep='\t', index_col='code')
del df1['Unnamed: 0']

df2 = pd.read_csv('../data/ipo_details', sep= '\t', index_col = 0)

#Use combine_first to avoid duplicate columns
#result = pd.concat([df, df2], axis = 1, join='inner')
df = df1.combine_first(df2)

#Save to file
df.to_csv('../data/ipo_data_processed', sep='\t', encoding='utf-8')
df.columns.values

array(['area', 'banks', 'buy_ratio', 'category', 'date', 'draw_prob',
       'eipo', 'firstday_performance', 'hk_portion', 'ipo_price',
       'ipo_price_range', 'market_type', 'name', 'now_price', 'one_hand',
       'predict_profile_market_ratio', 'predict_profit_ratio',
       'profit_ratio', 'recommender', 'sales', 'shares_per_hand',
       'stock_type', 'total_performance', 'total_value', 'website'], dtype=object)

In [3]:
#Now use dataframe and do some feature engineering
# Drop unrelated columns
to_del = ['date', 'banks', 'eipo', 'name', 'now_price', 'website', 'total_performance','predict_profile_market_ratio', 'predict_profit_ratio', 'profit_ratio']
for item in to_del:
    del df[item]

#Drop non_public ipo stocks
df = df[df.draw_prob.notnull()]


def buy_ratio_process(x):
    if x == '認購不足':
        return 0.0
    else:
        return float(x)

def hk_portion_process(x):
    if not pd.isnull(x):
        x = x.split('(')[1].strip(')')
        return x.strip()
    else:
        return x
    
def per2float(x):
    if not pd.isnull(x):
        x = x.strip('%')
        return float(x)/100.
    else:
        return x

def get_low_bound(x):
    if ',' in str(x):
        x = x.replace(',', '')
    try:
        if pd.isnull(x) or '-' not in x:
            return float(x)
        else:
            x = x.split('-')
            return float(x[0])
    except Exception as e:
        print e
        print x

def get_up_bound(x):
    if ',' in str(x):
        x = x.replace(',', '')
    try:
        if pd.isnull(x) or '-' not in x:
            return float(x)
        else:
            x = x.split('-')
            return float(x[1])
    except Exception as e:
        print e
        print x
        
def get_ipo_range_prop(x):
    if pd.isnull(x):
        return x
    low_bound = get_low_bound(x)
    up_bound = get_up_bound(x)
    return (up_bound-low_bound)*2/(up_bound+low_bound)

def get_total_value_mid(x):
    if pd.isnull(x):
        return x
    low_bound = get_low_bound(x)
    up_bound = get_up_bound(x)
    return (up_bound+low_bound)/2
    
#Format buy_ratio
df['buy_ratio'] = df['buy_ratio'].apply(buy_ratio_process)

#Format hk-porrtion
df['hk_portion'] = df['hk_portion'].apply(hk_portion_process)

#Format percentage
df['draw_prob'] = df['draw_prob'].apply(per2float)
df['firstday_performance'] = df['firstday_performance'].apply(per2float)
df['hk_portion'] = df['hk_portion'].apply(per2float)

#Split range to two columns
#df['ipo_price_low_bound'] = df['ipo_price_range'].apply(get_low_bound)
#df['ipo_price_up_bound'] = df['ipo_price_range'].apply(get_up_bound)
#del df['ipo_price_range']
#df['total_value_low_bound'] = df['total_value'].apply(get_low_bound)/1000000000.
#df['total_value_up_bound'] = df['total_value'].apply(get_up_bound)/1000000000.
#del df['total_value']


#Merge ipo_price_range to proportion of middle
df['ipo_price_range_ratio'] = df['ipo_price_range'].apply(get_ipo_range_prop)
del df['ipo_price_range']
df['total_value_mid'] = df['total_value'].apply(get_total_value_mid)/1000000000.
del df['total_value']


df

,area,buy_ratio,category,draw_prob,firstday_performance,hk_portion,ipo_price,market_type,one_hand,recommender,sales,shares_per_hand,stock_type,ipo_price_range_ratio,total_value_mid
code,,,,,,,,,,,,,,,
8465,全球,13.6,工業用品,1.000,0.0000,0.1000,1.68,香港聯合交易所有限公司 (創業板),1.0,工銀國際融資有限公司 (相關往績),工銀國際證券有限公司、創僑證券有限公司,2000.0,其他,0.372881,0.536900
8427,馬來西亞,11.2,電訊及網絡器材,1.000,1.5000,0.1000,0.28,香港聯合交易所有限公司 (創業板),1.0,興業金融融資有限公司 (相關往績),豐盛東方資本有限公司、潮商證券有限公司、智華證券有限公司,8000.0,其他,0.214286,0.173600
1216,中國,0.0,銀行,1.000,0.0163,0.0386,2.45,香港聯合交易所有限公司 (主板),1.0,中信里昂證券資本市場有限公司 (相關往績)、摩根大通證券(遠東)有限公司 (相關往績)、建銀...,中信里昂、摩根大通、建銀國際、招銀國際、交銀國際、廣發証券、中州國際、海通國際、浦銀國際、首控證券,1000.0,H股,0.044444,8.167500
8291,中國,12.1,鋼鐵,0.903,0.0462,0.5000,0.65,香港聯合交易所有限公司 (創業板),2.0,高企業融資有限公司 (相關往績),力高證券有限公司、擎天證券有限公司,5000.0,其他,0.333333,0.240000
8472,香港,10.5,其他支援服務,1.000,0.1000,0.1000,0.50,香港聯合交易所有限公司 (創業板),1.0,八方金融有限公司 (相關往績),興證國際金融集團有限公司、鴻鵬資本證券有限公司,5000.0,其他,0.000000,NaN
1676,中國,8.2,包裝食品,1.000,1.0000,0.1000,0.66,香港聯合交易所有限公司 (主板),1.0,東興證券(香港)有限公司 (相關往績),東興證券(香港)有限公司、富滙證券有限公司、雅利多證券有限公司,4000.0,其他,0.349650,0.715000
1649,中國,0.0,供電供熱,1.000,-0.0125,0.0494,1.60,香港聯合交易所有限公司 (主板),1.0,中國國際金融香港證券有限公司 (相關往績),中國國際金融香港證券有限公司、中銀國際亞洲有限公司,2000.0,H股,0.048780,1.262800
8481,全球,0.3,家具及家居用品,1.000,0.4265,0.1000,0.68,香港聯合交易所有限公司 (創業板),1.0,大有融資有限公司 (相關往績),阿仕特朗資本管理有限公司、鼎成證券有限公司、中國建信金融服務有限公司、創陞證券有限公司、華邦...,5000.0,其他,0.370370,0.337500
8462,新加坡,2.2,其他支援服務,1.000,0.0556,0.1000,0.45,香港聯合交易所有限公司 (創業板),1.0,創僑國際有限公司 (相關往績),太平基業證券有限公司,5000.0,其他,0.200000,0.300000


In [8]:
#Now do one-hot encoding for all categorical columns
#One problem is that we have to split('、') first for contents with multiple companies

dftest = df.copy()

def one_hot_encoding(df, column_name):
#Reads a df and target column, does tailored one-hot encoding, and return new df for merge

    cat_list = df[column_name].unique().tolist()
    cat_set = set()
    for items in cat_list:
        if pd.isnull(items):
            continue
        items = items.split('、')
        for item in items:
            item = item.strip()
            cat_set.add(item)
    for item in cat_set:
        item = column_name + '_' + item
        df[item] = 0
    
    def check_onehot(x, cat):
        if pd.isnull(x):
            return 0
        x = x.split('、')
        for item in x:
            if cat == item.strip():
                return 1
        return 0
    
    for item in cat_set:
        df[column_name + '_' + item] = df[column_name].apply(check_onehot, args=(item, ))
    
    del df[column_name]
    return df
    
dftest = one_hot_encoding(dftest, 'area')
dftest = one_hot_encoding(dftest, 'category')
dftest = one_hot_encoding(dftest, 'market_type')
dftest = one_hot_encoding(dftest, 'recommender')
dftest = one_hot_encoding(dftest, 'sales')
dftest = one_hot_encoding(dftest, 'stock_type')

dftest.to_csv('../data/hk_ipo_feature_engineered', sep='\t', encoding='utf-8')
dftest


,buy_ratio,draw_prob,firstday_performance,hk_portion,ipo_price,one_hand,shares_per_hand,ipo_price_range_ratio,total_value_mid,area_大中華,...,sales_招商(香港),sales_太平基業證券,sales_寶威證券有限公司,sales_中天證券有限公司,sales_中國光大證券(香港)有限公司,sales_中州國際,sales_瑞士信貸,stock_type_紅籌,stock_type_H股,stock_type_其他
code,,,,,,,,,,,,,,,,,,,,,
8465,13.6,1.000,0.0000,0.1000,1.68,1.0,2000.0,0.372881,0.536900,0,...,0,0,0,0,0,0,0,0,0,1
8427,11.2,1.000,1.5000,0.1000,0.28,1.0,8000.0,0.214286,0.173600,0,...,0,0,0,0,0,0,0,0,0,1
1216,0.0,1.000,0.0163,0.0386,2.45,1.0,1000.0,0.044444,8.167500,0,...,0,0,0,0,0,1,0,0,1,0
8291,12.1,0.903,0.0462,0.5000,0.65,2.0,5000.0,0.333333,0.240000,0,...,0,0,0,0,0,0,0,0,0,1
8472,10.5,1.000,0.1000,0.1000,0.50,1.0,5000.0,0.000000,NaN,0,...,0,0,0,0,0,0,0,0,0,1
1676,8.2,1.000,1.0000,0.1000,0.66,1.0,4000.0,0.349650,0.715000,0,...,0,0,0,0,0,0,0,0,0,1
1649,0.0,1.000,-0.0125,0.0494,1.60,1.0,2000.0,0.048780,1.262800,0,...,0,0,0,0,0,0,0,0,1,0
8481,0.3,1.000,0.4265,0.1000,0.68,1.0,5000.0,0.370370,0.337500,0,...,0,0,0,0,0,0,0,0,0,1
8462,2.2,1.000,0.0556,0.1000,0.45,1.0,5000.0,0.200000,0.300000,0,...,0,0,0,0,0,0,0,0,0,1
